In [1]:
from python_graphql_client import GraphqlClient
import datetime
import pandas
import IPython

# Instantiate the client with an endpoint.
clientv2 = GraphqlClient(endpoint="https://api.entur.io/journey-planner/v2/graphql")
clientv3 = GraphqlClient(endpoint="https://api.entur.io/journey-planner/v3/graphql")

#timestamp = "2021-09-01T12:20:00+02:00"
timestamp = datetime.datetime.now().astimezone().isoformat()

# Buran 1
#apiQuay = "NSR:Quay:103009"
#atbQuay = "Buran+1"

# Buran 2
#apiQuay = "NSR:Quay:103007"
#atbQuay = "Buran+2"

# Buran 3
api_quay = "NSR:Quay:103008"
atb_quay = "Buran+3"

# Valøyvegen
#api_quay = "NSR:Quay:71982"
#atb_quay = "Val%C3%B8yvegen"

# Create the query string and variables required for the request.
query = """
query ($id: String!, $timestamp: DateTime) {
  quay(id: $id) {
    name
    publicCode
    estimatedCalls(numberOfDepartures: 20, startTime: $timestamp) {
      expectedArrivalTime
      aimedArrivalTime
      actualArrivalTime
      actualDepartureTime
      destinationDisplay {
        frontText
      }
      serviceJourney {
        line {
          publicCode
        }
      }
    }
  }
}


"""

# 103009 = Buran
variables = {
    "id": api_quay,
    "timestamp": timestamp,
}

datav2 = clientv2.execute(query=query, variables=variables)
datav3 = clientv3.execute(query=query, variables=variables)

# datav3


# Quay info
"""
{
  quay(id: "NSR:Quay:103009") {
    id
    name
    stopPlace {
      name
      id
      quays {
        id
        name
        publicCode
      }
    }
    publicCode
    estimatedCalls {
      destinationDisplay {
        frontText
      }
      actualArrivalTime
      actualDepartureTime
      expectedArrival {
        time
      }
    }
  }
}
"""


'\n{\n  quay(id: "NSR:Quay:103009") {\n    id\n    name\n    stopPlace {\n      name\n      id\n      quays {\n        id\n        name\n        publicCode\n      }\n    }\n    publicCode\n    estimatedCalls {\n      destinationDisplay {\n        frontText\n      }\n      actualArrivalTime\n      actualDepartureTime\n      expectedArrival {\n        time\n      }\n    }\n  }\n}\n'

In [2]:
# API OUTPUT v2
quayv2 = datav2['data']['quay']
quayNamev2 = quayv2['name'] + " " + quayv2['publicCode']

callsv2 = datav2['data']['quay']['estimatedCalls']
api_outputv2 = []

for arrival in callsv2:
    expectedTime = arrival['expectedArrivalTime'].split('T')[1].split('+')[0]
    aimedTime = arrival['aimedArrivalTime'].split('T')[1].split('+')[0]
    lineNr = arrival['serviceJourney']['line']['publicCode']
    lineName = arrival['destinationDisplay']['frontText']
    api_outputv2.append([lineNr, lineName, expectedTime, aimedTime])
    
    
# API OUTPUT v3
quayv3 = datav3['data']['quay']
quayNamev3 = quayv3['name'] + " " + quayv3['publicCode']

callsv3 = datav3['data']['quay']['estimatedCalls']
api_outputv3 = []

for arrival in callsv3:
    expectedTime = arrival['expectedArrivalTime'].split('T')[1].split('+')[0]
    aimedTime = arrival['aimedArrivalTime'].split('T')[1].split('+')[0]
    lineNr = arrival['serviceJourney']['line']['publicCode']
    lineName = arrival['destinationDisplay']['frontText']
    api_outputv3.append([lineNr, lineName, expectedTime, aimedTime])

# sammenlikn med
# https://rp.atb.no/scripts/TravelMagic/TravelMagicWE.dll/svar?referrer=&lang=no&dep1=1&theme=&from=Buran+1+%28Trondheim%29&to=&Time=14%3A45&Date=30.08.2021&direction=1&search=Vis+reiseforslag&GetTR0=1&GetTR1=1&GetTR2=1&GetTR3=1&GetTR4=1&GetTR5=1&GetTR6=1&GetTR7=1&through=&throughpause=&changepenalty=1&changepause=0&linjer=&destinations=

# Set up atb.no capture
dt = datetime.datetime.fromisoformat(timestamp)
time = str(dt.hour) + "%3A" + str(dt.minute)
date = str(dt.day) + "." + str(dt.month) + "." + str(dt.year)

atb_url = "https://rp.atb.no/scripts/TravelMagic/TravelMagicWE.dll/svar?referrer=&lang=no&dep1=1&theme=&to="
atb_url = atb_url + "&from=" + atb_quay
atb_url = atb_url + "&Time=" + time
atb_url = atb_url + "&Date=" + date
atb_url = atb_url + "&direction=1&search=Vis+reiseforslag&GetTR0=1&GetTR1=1&GetTR2=1&GetTR3=1&GetTR4=1&GetTR5=1&GetTR6=1&GetTR7=1&through=&throughpause=&changepenalty=1&changepause=0&linjer=&destinations="



In [3]:

# Display results
pandas.DataFrame(api_outputv2, columns=[quayNamev2 + ' v2', "Linjetekst", "Estimert tid", "Planlagt tid"])

,Buran 3 v2,Linjetekst,Estimert tid,Planlagt tid
0,2,Strindheim via Lade,11:17:01,11:15:00
1,20,Grillstad via Ladeham.-Strindh.,11:20:00,11:20:00
2,2,Strindheim via Lade,11:27:02,11:25:00
3,2,Strindheim via Lade,11:36:46,11:35:00
4,2,Strindheim via Lade,11:46:47,11:45:00
5,20,Grillstad via Ladeham.-Strindh.,11:51:48,11:50:00
6,2,Strindheim via Lade,11:56:48,11:55:00
7,2,Strindheim via Lade,12:06:48,12:05:00
8,2,Strindheim via Lade,12:16:50,12:15:00
9,20,Grillstad via Ladeham.-Strindh.,12:21:53,12:20:00


In [4]:
# Display results
pandas.DataFrame(api_outputv3, columns=[quayNamev3 + ' v3', "Linjetekst", "Estimert tid", "Planlagt tid"])

,Buran 3 v3,Linjetekst,Estimert tid,Planlagt tid
0,2,Strindheim via Lade,11:17:01,11:15:00
1,20,Grillstad via Ladeham.-Strindh.,11:20:00,11:20:00
2,2,Strindheim via Lade,11:27:02,11:25:00
3,2,Strindheim via Lade,11:36:46,11:35:00
4,2,Strindheim via Lade,11:46:47,11:45:00
5,20,Grillstad via Ladeham.-Strindh.,11:51:48,11:50:00
6,2,Strindheim via Lade,11:56:48,11:55:00
7,2,Strindheim via Lade,12:06:48,12:05:00
8,2,Strindheim via Lade,12:16:50,12:15:00
9,20,Grillstad via Ladeham.-Strindh.,12:21:53,12:20:00


In [5]:
IPython.display.IFrame(atb_url, 500, 750)